In [15]:
import numpy as np 
import pandas as pd
import json
%pylab inline
import keras
import tensorflow as tf

Populating the interactive namespace from numpy and matplotlib


In [2]:
from keras import Model, Input
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.applications import InceptionV3,inception_v3

In [3]:
labels = json.load(open('data/reformatted_train.json'))

In [4]:
NUM_CLASSES = 228

In [5]:
numbers = [str(x) for x in range(1,10000)]
X,y = [],[]
for n in numbers:   
    img = image.load_img('data/initial_images/{}.jpg'.format(n),grayscale=False,target_size=(224,224,3))
    image.img_to_array(img)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = inception_v3.preprocess_input(x)
    X.append(x.reshape((224,224,3)))
    label = labels[n]['labelId']
    label = np.array([int(l) - 1 for l in label])
    label = keras.utils.to_categorical(label, num_classes = NUM_CLASSES).sum(axis = 0)
    y.append(label)
X,y = np.array(X), np.array(y)

In [6]:
from sklearn.model_selection import train_test_split

In [39]:
largest = 0
for l in y:
    for num in l:
        if num > largest:
            largest = num

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
def create_model(base_layers):
    for layer in base_layers.layers:
        layer.trainable = False
    X = base_layers.output
    X = Flatten()(X)
    X = Dense(1000, activation = 'relu', name = 'dense1')(X)
    predictions = Dense(NUM_CLASSES, activation = 'sigmoid', name = 'outlayer')(X)
    model = Model(inputs = base_layers.inputs, outputs = predictions)
    return model

In [8]:
inception_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3))

In [10]:
model1 = create_model(inception_base)

In [11]:
data_gen = ImageDataGenerator(shear_range= 0.3,
                              horizontal_flip= True,
                              width_shift_range=0.3,
                              height_shift_range = 0.3)

In [12]:
training_gen = data_gen.flow(X_train,y_train,seed = 42)

In [13]:
sgd = keras.optimizers.SGD(lr = .01, decay = .00001)
rmsprop = keras.optimizers.RMSprop(lr = .01, decay = .00001)

In [20]:
model1.compile(optimizer = rmsprop, loss =  keras.losses.binary_crossentropy , metrics = ['accuracy'])

In [21]:
model1.fit_generator(training_gen,
                     epochs  =40,
                     steps_per_epoch= 50, 
                     validation_data = (X_test,y_test),
                     workers = 6,
                     use_multiprocessing = True,
                     verbose = 1)

Epoch 1/40
50/50 [==============================] - 35s 694ms/step - loss: 2.0547 - acc: 0.8712 - val_loss: 2.0561 - val_acc: 0.8711
Epoch 2/40
50/50 [==============================] - 32s 631ms/step - loss: 2.0534 - acc: 0.8713 - val_loss: 2.0561 - val_acc: 0.8711
Epoch 3/40
50/50 [==============================] - 32s 632ms/step - loss: 2.0490 - acc: 0.8716 - val_loss: 2.0561 - val_acc: 0.8711
Epoch 4/40
50/50 [==============================] - 32s 631ms/step - loss: 2.0554 - acc: 0.8712 - val_loss: 2.0561 - val_acc: 0.8711
Epoch 5/40
49/50 [============================>.] - ETA: 0s - loss: 2.0594 - acc: 0.8709

Process ForkPoolWorker-26:
Process ForkPoolWorker-25:
Process ForkPoolWorker-28:
Process ForkPoolWorker-27:
Process ForkPoolWorker-29:
Traceback (most recent call last):
Process ForkPoolWorker-30:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/connor/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/connor/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/connor/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/connor/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/connor/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/connor/anaconda3/lib/pytho

KeyboardInterrupt: 

In [25]:
(y_train == 0).mean()

0.9746404704096784